---
title: 9.7 Transformer
date: 2024-7-9 12:00:00
tags: [机器学习,pytorch]
categories: [机器学习]
comment: true
toc: true
---
#  
<!--more-->
# 7 Transformer

- 自注意力同时具有并行计算和最短和最大路径长度这两个优势。

- 对比之前仍然依赖rnn实现输入表示的自注意力模型，Transformer完全基于注意力机制，没有任何卷积层或rnn层。

## 7.1 模型

- 编码器-解码器架构。
![](../../../../../../themes/yilia/source/img/deeplearning/code/pytorch/9_attention/7_transformer/1.jpg)
![](img/deeplearning/code/pytorch/9_attention/7_transformer/1.jpg)

    - 编码器由多个层叠加，每个层有两个子层（sublayer）
        - 第一个子层是multi-head self-attention汇聚；
        - 第二个子层是基于位置的前馈网络（positionwise feed-forward network）
        - 在计算编码器的注意力时，q,k,v都来自前一个编码器层的输出。输入序列对应的每个位置，Transformer编码器都将输出一个d维表示向量

    - 解码器多了一个子层：encoder-decoder attention层。其中q来自前一个解码器层的输出，但k,v来自整个编码器的输出。
    - 在解码器注意力中，q,k,v都来自撒谎给你一个解码器层的输出。但是解码器中的每一个位置只能考虑该位置之前的所有位置。这种masked attention保留了自回归（auto-regressive）属性，确保预测仅依赖于已生成的输出词元。

In [1]:
import math
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

## 7.2 基于位置的前馈网络

- 基于位置的前馈网络对序列中所有位置的表示进行变换时 使用的是同一个mlp，因此称前馈网络是基于位置的。
- (batch, n, hidden) -> (batch, n, output)

In [19]:
#@save
class PositionWiseFFN(nn.Module):
    '''基于位置的前馈网络'''
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs, **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))



- 因为使用同一个多层感知机对所有位置上的输入进行变换，所以当这些位置的输入相同时，输出相同：
    - 输入3个序列，特征长度4；由于输入的三个序列一样，所以输出的三个序列也一样。

In [20]:
ffn = PositionWiseFFN(4,4,8)
ffn.eval()
ffn(torch.ones((2,3,4)))[0]

tensor([[-0.1422,  0.4383, -0.2341,  0.4985, -0.4600,  0.2172,  0.7684,  0.1997],
        [-0.1422,  0.4383, -0.2341,  0.4985, -0.4600,  0.2172,  0.7684,  0.1997],
        [-0.1422,  0.4383, -0.2341,  0.4985, -0.4600,  0.2172,  0.7684,  0.1997]],
       grad_fn=<SelectBackward0>)

## 7.3 残差连接和层规范化

- 层规范化和批量规范化目标相同
    - 层规范化：基于特征维度进行规范化
- 尽管批量规范化在cv中广泛应用，但nlp中（输入通常是变长序列）层规范化更好。

- 对比不同维度的层规范化和批量规范化的效果：(一个横着归一化，一个竖着归一化)

In [26]:
ln = nn.LayerNorm(2)
bn = nn.BatchNorm1d(2)
X = torch.tensor([[1,2],[2,3]], dtype=torch.float32)
# 训练模式下计算X的均值和方差
print('layer norm:',ln(X), '\nbatch norm:',bn(X))

layer norm: tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>) 
batch norm: tensor([[-1.0000, -1.0000],
        [ 1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>)


- 使用残差连接和层规范化：

In [27]:
#@save
class AddNorm(nn.Module):
    '''残差连接后进行层规范化'''
    def __init__(self, normalized_shape, dropout, **kwargs) -> None: #不用输入batch维度
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)
    
add_norm = AddNorm([3,4], 0.5)
add_norm.eval()
add_norm(torch.ones((2,3,4)), torch.ones((2,3,4))).shape

torch.Size([2, 3, 4])

## 7.4 编码器

In [ ]:
#@save
class EncoderBlock(nn.Module):
    '''transformer编码器块'''
    def __init__(self, key_size, query_size, value_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout, use_bias
        )
        self.addnorm1 = AddNorm(norm_shape, dropout)

        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X,self.attention(X,X,X,valid_lens))
        return self.addnorm2(Y,self.ffn(Y))